# PROBLEM ONE: Route Opitimization

#### FULL SERVICE WITH 737 MAX

In [3]:
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xlrd

data1 = pd.read_csv('routes.csv')
data1.head()
data = data1.values

route,start, finish, distance, cost, flight_time, demand, rev = multidict({item[0]: (item[1],item[2],item[3],item[4],item[5],item[6],item[7]) for item in data1.values})

headers = data1.columns.values[1:8]

min_week = 10800
plane_efficiency = 0.9
total_airtime = min_week*plane_efficiency

Qx = 5 #number of 777
Qy = 10 # number of 787
Qz = 5 #number of 737

Sx = 350 #capcities of 777,787 and 737
Sy = 250
Sz = 190

Cx = 1.5 #cost multiplier of 777,787 and 737
Cy = 1.2
Cz = 1


m= Model('route_network')

x= m.addVars(route,vtype=GRB.INTEGER,lb=0, name = '777') # no. of routes i flown using 777, i = sg to maninila, sg to tokyo,etc
y= m.addVars(route,vtype=GRB.INTEGER,lb=0, name = '787') # using 787
z= m.addVars(route,vtype=GRB.INTEGER,lb=0, name = '737') #using 737 max

m.setObjective(quicksum((Sx*x[i]+Sy*y[i]+Sz*z[i])*rev[i] for i in route)-quicksum((Cx*x[i]+Cy*y[i]+Cz*z[i])*cost[i] for i in route), GRB.MAXIMIZE)

# planes can only be in the air 90% of the time per week

m.addConstr(quicksum(x[i]*flight_time[i] for i in route) <= total_airtime*Qx*0.5)
m.addConstr(quicksum(y[i]*flight_time[i] for i in route) <= total_airtime*Qy*0.5)
m.addConstr(quicksum(z[i]*flight_time[i] for i in route) <= total_airtime*Qz*0.5)

#planes only fly if there is demand, demand is assumed to be known and that planes do not fly more seats than there is demand

m.addConstrs((Sx*x[i]+Sy*y[i]+Sz*z[i]) <= demand[i] for i in route)

#737 max has limited range

m.addConstrs((distance[i]*z[i]) <= z[i]*3500 for i in route)

#all routes must start from Singapore

m.addConstr(x[route[1]] >= x[route[7]]+ x[route[8]] + x[route[9]])
m.addConstr(x[route[2]] >= x[route[10]]+x[route[11]]+ x[route[12]])
m.addConstr(x[route[3]] >= x[route[13]]+x[route[14]]+ x[route[15]])
m.addConstr(x[route[4]] >= x[route[16]]+x[route[17]]+ x[route[18]] + x[route[19]])
m.addConstr(x[route[5]] >= x[route[20]]+x[route[21]]+ x[route[22]] + x[route[23]])
m.addConstr(x[route[6]] >= x[route[24]]+x[route[25]]+ x[route[26]] + x[route[27]] + x[route[28]])

m.addConstr(y[route[1]] >= y[route[7]]+ y[route[8]] + y[route[9]])
m.addConstr(y[route[2]] >= y[route[10]]+y[route[11]]+ y[route[12]])
m.addConstr(y[route[3]] >= y[route[13]]+y[route[14]]+ y[route[15]])
m.addConstr(y[route[4]] >= y[route[16]]+y[route[17]]+ y[route[18]] + y[route[19]])
m.addConstr(y[route[5]] >= y[route[20]]+y[route[21]]+ y[route[22]] + y[route[23]])
m.addConstr(y[route[6]] >= y[route[24]]+y[route[25]]+ y[route[26]] + y[route[27]] + y[route[28]])

m.addConstr(x[route[1]] >= z[route[7]]+ z[route[8]] + z[route[9]])
m.addConstr(x[route[2]] >= z[route[10]]+z[route[11]]+ z[route[12]])
m.addConstr(x[route[3]] >= z[route[13]]+z[route[14]]+ z[route[15]])
m.addConstr(x[route[4]] >= z[route[16]]+z[route[17]]+ z[route[18]] + z[route[19]])
m.addConstr(x[route[5]] >= z[route[20]]+z[route[21]]+ z[route[22]] + z[route[23]])
m.addConstr(x[route[6]] >= z[route[24]]+z[route[25]]+ z[route[26]] + z[route[27]] + z[route[28]])

# 0 to 6: routes from sg
#7:9 routes from manila
#10:12 routes from beijing
#13:15 routes from  tokyo
#16:29 routes from abu dhabi
#20:23 routes from johannesburg
#24:28 outes from kl

m.optimize()

# print optimal solutions
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))
    
#print optimal value
print('Obj: %g' % m.objVal)


Optimize a model with 79 rows, 87 columns and 287 nonzeros
Variable types: 0 continuous, 87 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+02, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 1e+05]
Found heuristic solution: objective 6757128.3349
Presolve removed 56 rows and 50 columns
Presolve time: 0.00s
Presolved: 23 rows, 37 columns, 89 nonzeros
Variable types: 0 continuous, 37 integer (0 binary)

Root relaxation: objective 1.346508e+07, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.3465e+07    0    7 6757128.33 1.3465e+07  99.3%     -    0s
H    0     0                    1.338046e+07 1.3465e+07  0.63%     -    0s
H    0     0                    1.340301e+07 1.3465e+07  0.46%     -    0s
H    0     0                    1.342260e+07 1.3465e+07  0.32%     -    0s
 

#### PARTIAL SERVICE WITH NO 737 MAX 

In [219]:
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xlrd

data1 = pd.read_csv('routes.csv')
data1.head()
data = data1.values

route,start, finish, distance, cost, flight_time, demand, rev = multidict({item[0]: (item[1],item[2],item[3],item[4],item[5],item[6],item[7]) for item in data1.values})

headers = data1.columns.values[1:8]

min_week = 10800
plane_efficiency = 0.9
total_airtime = min_week*plane_efficiency

Qx = 5 #number of 777
Qy = 10 # number of 787
Qz = 0 #number of 737

Sx = 350 #capcities of 777,787 and 737
Sy = 250
Sz = 190

Cx = 1.5 #cost multiplier of 777,787 and 737
Cy = 1.2
Cz = 1


m= Model('route_network')

x= m.addVars(route,vtype=GRB.INTEGER,lb=0, name = '777') # no. of routes i flown using 777, i = sg to maninila, sg to tokyo,etc
y= m.addVars(route,vtype=GRB.INTEGER,lb=0, name = '787') # using 787
z= m.addVars(route,vtype=GRB.INTEGER,lb=0, name = '737') #using 737 max

m.setObjective(quicksum((Sx*x[i]+Sy*y[i]+Sz*z[i])*rev[i] for i in route)-quicksum((Cx*x[i]+Cy*y[i]+Cz*z[i])*cost[i] for i in route), GRB.MAXIMIZE)

# planes can only be in the air 90% of the time per week

m.addConstr(quicksum(x[i]*flight_time[i] for i in route) <= total_airtime*Qx*0.5)
m.addConstr(quicksum(y[i]*flight_time[i] for i in route) <= total_airtime*Qy*0.5)
m.addConstr(quicksum(z[i]*flight_time[i] for i in route) <= total_airtime*Qz*0.5)

#planes only fly if there is demand, demand is assumed to be known and that planes do not fly more seats than there is demand

m.addConstrs((Sx*x[i]+Sy*y[i]+Sz*z[i]) <= demand[i] for i in route)

#737 max has limited range

m.addConstrs((distance[i]*z[i]) <= z[i]*3500 for i in route)

#all routes must start from Singapore

m.addConstr(x[route[1]] >= x[route[7]]+ x[route[8]] + x[route[9]])
m.addConstr(x[route[2]] >= x[route[10]]+x[route[11]]+ x[route[12]])
m.addConstr(x[route[3]] >= x[route[13]]+x[route[14]]+ x[route[15]])
m.addConstr(x[route[4]] >= x[route[16]]+x[route[17]]+ x[route[18]] + x[route[19]])
m.addConstr(x[route[5]] >= x[route[20]]+x[route[21]]+ x[route[22]] + x[route[23]])
m.addConstr(x[route[6]] >= x[route[24]]+x[route[25]]+ x[route[26]] + x[route[27]] + x[route[28]])

m.addConstr(y[route[1]] >= y[route[7]]+ y[route[8]] + y[route[9]])
m.addConstr(y[route[2]] >= y[route[10]]+y[route[11]]+ y[route[12]])
m.addConstr(y[route[3]] >= y[route[13]]+y[route[14]]+ y[route[15]])
m.addConstr(y[route[4]] >= y[route[16]]+y[route[17]]+ y[route[18]] + y[route[19]])
m.addConstr(y[route[5]] >= y[route[20]]+y[route[21]]+ y[route[22]] + y[route[23]])
m.addConstr(y[route[6]] >= y[route[24]]+y[route[25]]+ y[route[26]] + y[route[27]] + y[route[28]])

m.addConstr(x[route[1]] >= z[route[7]]+ z[route[8]] + z[route[9]])
m.addConstr(x[route[2]] >= z[route[10]]+z[route[11]]+ z[route[12]])
m.addConstr(x[route[3]] >= z[route[13]]+z[route[14]]+ z[route[15]])
m.addConstr(x[route[4]] >= z[route[16]]+z[route[17]]+ z[route[18]] + z[route[19]])
m.addConstr(x[route[5]] >= z[route[20]]+z[route[21]]+ z[route[22]] + z[route[23]])
m.addConstr(x[route[6]] >= z[route[24]]+z[route[25]]+ z[route[26]] + z[route[27]] + z[route[28]])

# 0 to 6: routes from sg
#7:9 routes from manila
#10:12 routes from beijing
#13:15 routes from  tokyo
#16:29 routes from abu dhabi
#20:23 routes from johannesburg
#24:28 outes from kl

#need to maintain route from sg to tokyo and sg to abu dhabi, these routes were flow by the 737 before the groundings

m.addConstr(x[route[3]] + y[route[3]] >= 15) # in this case, the airline wants at least 15 weekly flighs to both destinations
m.addConstr(x[route[4]] + y[route[4]] >= 15)

m.optimize()

# print optimal solutions
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))
    
#print optimal value
print('Obj: %g' % m.objVal)


Optimize a model with 81 rows, 87 columns and 291 nonzeros
Variable types: 0 continuous, 87 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+02, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+05]
Found heuristic solution: objective 6851865.0026
Presolve removed 60 rows and 55 columns
Presolve time: 0.00s
Presolved: 21 rows, 32 columns, 80 nonzeros
Variable types: 0 continuous, 32 integer (0 binary)

Root relaxation: objective 1.236811e+07, 9 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.2368e+07    0    5 6851865.00 1.2368e+07  80.5%     -    0s
H    0     0                    1.226792e+07 1.2368e+07  0.82%     -    0s
H    0     0                    1.233341e+07 1.2368e+07  0.28%     -    0s
H    0     0                    1.235277e+07 1.2368e+07  0.12%     -    0s
  

# PROBLEM TWO: Seating Configuration

In [220]:
df=pd.read_csv("demand.csv")

FileNotFoundError: File b'demand.csv' does not exist

In [ ]:
Firstclassmean=df['First Class'].mean()
Businessclassmean=df['Business Class'].mean()
EconomyClassmean=df['Economy Class'].mean()

In [ ]:
# Create a new model

m = Model("Boeing_777_AirlineSeats")

# Create variables
x1 = m.addVar(name="First Class Seat")
x2 = m.addVar(name="Business Class Seat")
x3 = m.addVar(name="Economy Class Seat")
    
# Set objective
obj=3499*x1+ 900*x2+ 550*x3
m.setObjective(obj, GRB.MAXIMIZE)

# Add constraint: 
m.addConstr(25*x1+9*x2+3*x3<=2500) #weight
m.addConstr(35*x1 + 25*x2 + 15*x3 <=7705) #space
m.addConstr(x1<=Firstclassmean) #average demand
m.addConstr(x2<=Businessclassmean)
m.addConstr(x3<=EconomyClassmean)

m.optimize()

# print optimal solutions
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))
    
# print optimal value
print('Obj: %g' % m.objVal)

In [ ]:
# Create a new model

m = Model("Boeing_787_AirlineSeats")

# Create variables
x1 = m.addVar(name="First Class Seat")
x2 = m.addVar(name="Business Class Seat")
x3 = m.addVar(name="Economy Class Seat")
    
# Set objective
obj=3499*x1+ 900*x2+ 550*x3
m.setObjective(obj, GRB.MAXIMIZE)

# Add constraint: 
m.addConstr(25*x1+9*x2+3*x3<=1750) #weight
m.addConstr(35*x1 + 25*x2 + 15*x3 <=5505) #space
m.addConstr(x1<=Firstclassmean) #average demand
m.addConstr(x2<=Businessclassmean)
m.addConstr(x3<=EconomyClassmean)

m.optimize()

# print optimal solutions
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))
    
# print optimal value
print('Obj: %g' % m.objVal)

In [ ]:
# Create a new model

m = Model("Boeing_737Max_AirlineSeats")

# Create variables
x1 = m.addVar(name="First Class Seat")
x2 = m.addVar(name="Business Class Seat")
x3 = m.addVar(name="Economy Class Seat")
    
# Set objective
obj=3499*x1+ 900*x2+ 550*x3
m.setObjective(obj, GRB.MAXIMIZE)

# Add constraint: 
m.addConstr(25*x1+9*x2+3*x3<=1350) #weight
m.addConstr(35*x1 + 25*x2 + 15*x3 <=4165) #space
m.addConstr(x1<=Firstclassmean) #average demand
m.addConstr(x2<=Businessclassmean)
m.addConstr(x3<=EconomyClassmean)

m.optimize()

# print optimal solutions
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))
    
# print optimal value
print('Obj: %g' % m.objVal)

# PROBLEM THREE: Worker Scheduling 

### Load Packages

In [ ]:
#packages
import numpy as np
from gurobipy import *
import os
import pandas as pd

### Import csv

In [ ]:
#Import csv
os.chdir('C:\\Users\\Jerrell\\Desktop\\DBA3701 - Optimisation\\Opti proj')
os.getcwd()

employee_list = pd.read_csv("employee_list.csv")  #read employee details from csv



### Create Variables

In [ ]:
#Create workers list from csv
workerList = employee_list['Name'].tolist()

#Create management list
mgm = employee_list.loc[employee_list['Rank'] == "Manager"]
mgmtList = mgm['Name'].tolist()             
nonmgmtList = [x for x in workerList if x not in mgmtList]

#cost of each shift of each worker
regCost = employee_list['Pay'].tolist()     

# Cost of overtime shift
OTCost = [1.5*x for x in regCost]

regularCost  = { w : regCost[i] for i,w in enumerate(workerList) }
overtimeCost  = { w : OTCost[i] for i,w in enumerate(workerList) }

### Create Shifts

In [ ]:
shiftList = ['Monday1','Monday2','Tuesday1','Tuesday2','Wednesday1','Wednesday2'
             ,'Thursday1','Thursday2','Friday1','Friday2','Saturday1','Saturday2','Sunday1','Sunday2']

shiftReq = [3,2,4,4,5,4,5,4,2,4,5,4,3,5]
shiftRequirements  = { s : shiftReq[i] for i,s in enumerate(shiftList) }

# Range of shifts that every workers is required to stay between
minShifts = 3
maxShifts = 7

# Number of shifts to trigger overtime
OTTrigger = 5

### Availability Schedule

In [ ]:
#Assumming everyone is available
availability = pd.DataFrame(np.ones((len(workerList), len(shiftList))), index=workerList, columns=shiftList)

# For illustration, assume following people are unavailable for certain shifts
availability.at['Anna','Tuesday1'] = 0
availability.at['Tom','Saturday2'] = 0
availability.at['Jake','Thursday1'] = 0

# Create availability dictionary
avail = {(w,s) : availability.loc[w,s] for w in workerList for s in shiftList}

## Model

In [4]:
model = Model("Workers Scheduling")

# ub ensures that workers are only staffed when they are available
x = model.addVars(workerList, shiftList, ub=avail, vtype=GRB.BINARY, name='x')


regHours = model.addVars(workerList, name='regHrs')
overtimeHours = model.addVars(workerList, name='overtimeHrs')
overtimeTrigger = model.addVars(workerList, name = "OT_Trigger", vtype = GRB.BINARY)



# Ensure proper number of workers are scheduled

shiftReq = model.addConstrs((
    (x.sum('*',s) == shiftRequirements[s] for s in shiftList)
), name='shiftRequirement')


# Differentiate between regular time and overtime

## Decompose total shifts for each worker into regular shifts and OT shifts
regOT1 = model.addConstrs((regHours[w] + overtimeHours[w] == x.sum(w,'*') for w in workerList))
## Ensure that regular shifts are accounted for first for each employee before counting OT shifts
regOT2 = model.addConstrs((regHours[w] <= OTTrigger for w in workerList))
## Only allow the OT trigger to come on when regular shift count is greater than regular shift limit
regOT3 = model.addConstrs((regHours[w] / OTTrigger >= overtimeTrigger[w] for w in workerList))



# Ensure each worker stays within min and max shift bounds

minShiftsConstr = model.addConstrs(((
    x.sum(w,'*') >= minShifts for w in workerList)
), name='minShifts')

maxShiftsConstr = model.addConstrs(((
    x.sum(w,'*') <= maxShifts for w in workerList)
), name='maxShifts')


# Ensure every shift has at least one manager

for s in shiftList:
    model.addConstr((quicksum(x.sum(m,s) for m in mgmtList) >= 1), name='mgmtStaffing'+str(s))



# Minimize total cost, accounting for pay difference between regular time and overtime

model.ModelSense = GRB.MINIMIZE

Cost = 0
Cost += (quicksum(regularCost[w]*regHours[w] + overtimeCost[w]*overtimeHours[w] for w in workerList))

model.setObjective(Cost)

model.optimize()

NameError: name 'workerList' is not defined